In [4]:
import os, csv, sys, re, gzip
import requests
from bs4 import BeautifulSoup
import pandas as pd


Research Ideas
* How does weather affect performance
** Performance on the day of the event
** In the 4 months prior to the start of the event
* Are there natural time clusters in marathon performance data (i.e. more people clustered around hours or half hours, etc.)
* Are people getting faster?  Can we back out the introduction of things like carbon plated running shoes from th data?
* Which US state has the fastest runners? 2021, 2022, 2023, based on where they ran versus where they are from?

Great reference article: https://runrepeat.com/state-of-running

Website to get marathon results from: https://www.marathonguide.com/results/browse.cfm?Year=2024

In [26]:
# Get a mandate of Set[(racename, date, location)]
import requests
from bs4 import BeautifulSoup

def get_race_names(url):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

         # Find all the race elements
        race_elements = soup.find_all('a', class_='resultsLinksList')
        
        # Extract race names and dates
        race_info = []
        for element in race_elements:
            race_container = element.parent
            race_string = race_container.get_text()
            # Define the regular expression pattern to match the race name and date
            pattern = r'([^\n]+)\n.+\s+(\d{1,2}/\d{1,2}/\d{2})'

            # Search for the pattern in the text
            match = re.search(pattern, race_string)
            if match:
                race_name = match.group(1)
                race_date = match.group(2)
                # Find the race name
                race_url = 'https://www.marathonguide.com/results/'+element.get('href')
                race_info.append((race_name, race_date, race_url))

        return race_info
        # Find all the race names
        race_names = soup.find_all('a', class_='resultsLinksList')
        
        # Extract text from the race name elements
        race_names = [race.get_text() for race in race_names]
        
        return race_names
    else:
        print("Failed to retrieve the webpage")
        return []
        
race_mandate_output_file = "race_mandates.csv"
with open (race_mandate_output_file+".temp", "w") as w_handle:
    writer = csv.writer(w_handle)
    writer.writerow(["Race Name","Date","Results URL"])

    for year in range(2000, 2024):
        url = "https://www.marathonguide.com/results/browse.cfm?Year=%s"%year
        race_names = get_race_names(url)
        for race in race_names:
            writer.writerow(race)
os.rename(race_mandate_output_file+".temp", race_mandate_output_file)
print("Race Mandates: %s"%race_mandate_output_file)

In [29]:
# For a given mandate of (race name, date, url) get all of the available result pages
def get_result_pages(url):
     # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        result_rows = soup.find_all('tr')
        print(result_rows)
        
    else:
        print("Failed to retrieve the webpage")
        return []

url = "https://www.marathonguide.com/results/browse.cfm?RL=1&MIDD=560001217&Gen=B&Begin=1&End=64&Max=64"
get_result_pages(url)

[<tr>
<td>
<a class="Internallinks" href="index.cfm">Results Home</a>
<br/><b>Christmas Marathon:</b>
<a class="Internallinks" href="../races/racedetails.cfm?MIDD=560001217">Information &amp; Reviews</a> |
		
			<a class="Internallinks" href="../news/search.cfm?M_ID=560">News</a> |
		
		<br/><b>Christmas Marathon Results:</b>
<a class="Internallinks" href="../results/browse.cfm?MIDD=560101218">2010</a> |
		
		<a class="Internallinks" href="../results/browse.cfm?MIDD=560061223">2006</a> |
		
		<a class="Internallinks" href="../results/browse.cfm?MIDD=560051218">2005</a> |
		
		<a class="Internallinks" href="../results/browse.cfm?MIDD=560041219">2004</a> |
		
		<a class="Internallinks" href="../results/browse.cfm?MIDD=560031221">2003</a> |
		
		<a class="Internallinks" href="../results/browse.cfm?MIDD=560021222">2002</a> |
		
		<a class="Internallinks" href="../results/browse.cfm?MIDD=560011223">2001</a> |
		
		<a class="Internallinks" href="../results/browse.cfm?MIDD=560001217">2000</a>

In [71]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time
import gzip

# Set Chrome options to run headlessly
chrome_options = Options()
chrome_options.add_argument("--headless")  # Set headless mode
# Path to the ChromeDriver executable
chrome_driver_path = '/usr/bin/chromedriver'

def get_result_pages(race_url, driver, page_sleep_timer_sec = 2):
    # f(race_url, driver) => Seq[Soup]
    # Example usage: Open a website and print its title
    driver.get(race_url)
     # Find the "Overall" dropdown menu
    overall_dropdown = driver.find_element(By.NAME, "RaceRange")
    #print(overall_dropdown.get_attribute("outerHTML"))
    # Select the second option in the dropdown menu (the first is a placeholder) 
    select = Select(overall_dropdown)
    select.select_by_index(1)
    
    # Find the submit button and click it
    submit_button = driver.find_element(By.NAME, "SubmitButton")
    submit_button.click()
    # Wait for the page to load (you might need to adjust the wait time)
    # TODO dynamically set the wait time based on looking up some element on the page and sleeping
    time.sleep(page_sleep_timer_sec)
    sources = [driver.page_source]
    # Find the element with a particular src attribute value

    def safe_check_for_more_results():
        try:
            more_results_arrow = driver.find_element(By.CSS_SELECTOR, '[src="../images/smallarrow_right.gif"]')
            return more_results_arrow
        except NoSuchElementException:
            return None
            
    element_with_src = safe_check_for_more_results()
    while(element_with_src):
        element_with_src.click()
        # Wait for the page to load (you might need to adjust the wait time)
        time.sleep(page_sleep_timer_sec)
        sources.append(driver.page_source)
        element_with_src = safe_check_for_more_results()
    
    return sources

#Create a Service object
service = Service(chrome_driver_path)
driver = None
page_cache = "temp_page_cache"
try:
    # Initialize Chrome WebDriver with options and service
    driver = webdriver.Chrome(service=service, options=chrome_options)
    with open('race_mandates.csv') as r_handle:
        reader = csv.DictReader(r_handle)
        rows = list(reader)
        rows.reverse() # start with the more recent races, go backwards in time
        for result in rows:
            clean_name = result["Race Name"].replace(" ","_").lower()
            clean_date = result["Date"].replace("/","_")
            cache_location = os.path.join(page_cache, clean_name, clean_date)
            if os.path.exists(cache_location):
                #print("Skipping %s on %s"%(clean_name, clean_date))
                continue
            else:
                try:
                    page_sources = get_result_pages(result["Results URL"], driver)
                    
                    os.makedirs(cache_location)
                    for n in range(0,len(page_sources)):
                        with gzip.open(os.path.join(cache_location,"%s.tmp"%n), 'wb') as f:
                            f.write(page_sources[n].encode('utf-8'))
                except:
                    print("Unable to handle: %s for %s"%(clean_name, clean_date))
                        #with open(os.path.join(cache_location,"%s.tmp"%n), "w") as w_handle:
                    #    w_handle.write(page_sources[n])
    
    #race_url = "https://www.marathonguide.com/results/browse.cfm?MIDD=493240407"
    #page_sources = get_result_pages(race_url, driver)
    #for n in range(0,len(page_sources)):
    #    with open(os.path.join("temp","%s.tmp"%n), "w") as w_handle:
    #        w_handle.write(page_sources[n])
    #print(len(page_sources))
    
    # Note, this works - some thoughts/todos from here:
    # * Save page source, page to the next results, and repeat, until there is none
    # * For each page source load the source into beautiful soup
    # * Extract the  runners, their time, city/state/country, BQ status
    # * Refactor this into a function like get_runner_pages and get_runners_from_page
    # * Store in file with race name and year
finally:    
    # Close the WebDriver session
    if driver: driver.quit()

Unable to handle: grandfather_mountain_marathon for 7_10_21
Unable to handle: grandfather_mountain_marathon for 7_10_21
Unable to handle: sunflower_trail_marathon for 5_4_19
Unable to handle: riverboat_series_-_ky for 4_11_19
Unable to handle: grandfather_mountain_marathon for 7_13_19
Unable to handle: riverboat_series_-_ky for 4_11_19
Unable to handle: sunflower_trail_marathon for 5_4_19
Unable to handle: grandfather_mountain_marathon for 7_13_19
Unable to handle: riverboat_series_-_ky for 4_19_18
Unable to handle: grandfather_mountain_marathon for 7_14_18
Unable to handle: riverboat_series_-_ky for 4_19_18
Unable to handle: grandfather_mountain_marathon for 7_14_18
Unable to handle: riverboat_series_-_ky for 4_14_17
Unable to handle: grandfather_mountain_marathon for 7_8_17
Unable to handle: riverboat_series_-_ky for 4_14_17
Unable to handle: grandfather_mountain_marathon for 7_8_17
Unable to handle: riverboat_series_-_ky for 4_21_16
Unable to handle: grandfather_mountain_marathon fo

In [111]:
import concurrent.futures

def iterate_through_temp_cache(func, take=None, parallel=False):
    count = 0
    overall_results = []
    results = []
    # Note, this is not really that great, given that much of the operation is i/o bound...
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        futures = []
        for race_path in os.listdir("temp_page_cache"):
            if take and count > take: break
            # Iterate over directories to get race name, then sub directories to get date, 
            # and then files in directorises to get results
            for date in os.listdir(os.path.join("temp_page_cache", race_path)):
                if take and count > take: break
                count += 1
                race_cache_path = os.path.join("temp_page_cache", race_path, date)
                file_parts = [os.path.join(race_cache_path, f) for f in os.listdir(race_cache_path) if f.endswith("tmp")] 
                if not parallel: results.append(func(race_cache_path, file_parts))
                else:
                    futures.append(executor.submit(lambda x,y: func(x,y), race_cache_path, file_parts))

        for future in concurrent.futures.as_completed(futures):
            try:
                results.append(future.result())
            except Exception as exc:
                print(f'Processing data generated an exception: {exc}')
        return results

def extract_table_header(race_path, file_parts):
    headers = []
    #print("Processing %s"%race_path)
    for part in file_parts:
        with gzip.open(part) as r_handle:
            source = r_handle.read().decode('utf-8')
            soup = BeautifulSoup(source)
            possible_header_rows = soup.find_all('tr')
            for row in possible_header_rows:
                row_data = [cell.get_text(strip=True) for cell in row.find_all('th')]
                if len(row_data) > 0:
                    headers.append(row_data)
                    break
    return headers

def enrich_with_table_headers(race_path, file_parts):
    table_headers = extract_table_header(race_path, file_parts)
    if len(table_headers) > 0:
        with open(os.path.join(race_path, "headers.txt"), "w") as w_handle:
            writer = csv.writer(w_handle)
            writer.writerow(table_headers[0])
            #w_handle.write(",".join(["'"+x+"'" for x in table_headers[0]]))
    

iterate_through_temp_cache(enrich_with_table_headers, parallel=True)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [141]:
# Analyze Headers
# Collect all headers into data frame, sort, and determine how to parse

header_concordance = dict()
total_headers_parsed = 0
# Iterate through the directory and its subdirectories
for root, dirs, files in os.walk('temp_page_cache'):
    # Check if "headers.txt" exists in the list of files
    if "headers.txt" in files:
        total_headers_parsed += 1
        # Print the full path of the file
        with open(os.path.join(root, "headers.txt")) as handle:
            headers = handle.read().split('\'')
            headers = [h for h in headers if h != '' and h!=',']
            for header in headers:
                if header not in header_concordance:
                    header_concordance[header] = 0
                header_concordance[header] += 1
print(header_concordance)

def filter_to_headers_with(required_fields=[], optional_fields=[]):
    races = []
    for root, dirs, files in os.walk('temp_page_cache'):
        # Check if "headers.txt" exists in the list of files
        if "headers.txt" in files:
            # Print the full path of the file
            with open(os.path.join(root, "headers.txt")) as handle:
                headers = handle.read().split('\'')
                headers = set([h for h in headers if h != '' and h!=','])

                missing_required_field = False
                for field in required_fields:
                    if field not in headers:
                        missing_required_field = True
                        break

                has_optional_field = len(optional_fields) == 0
                for field in optional_fields:
                    if field in headers:
                        has_optional_field = True

                if not missing_required_field and has_optional_field:
                    races.append([root] + list(headers))
    return races



selection = filter_to_headers_with(['Last Name, First Name (Sex/Age)','Time'], ['City, State, Country','City, Country','City, State'])
                    
        


{'Last Name, First Name (Sex/Age)': 14153, 'Time': 13635, 'OverAllPlace': 14153, 'Sex Place/Div Place': 10808, 'DIV': 10828, 'Net Time': 5489, 'BQ*': 10725, 'City, State, Country': 8503, 'AG Time*': 9435, 'City': 654, 'Sex Place': 3343, 'City, Country': 124, 'State, Country': 276, 'City, State': 88, 'Country': 195, 'Div Place': 2, 'State': 16, 'Last Name, First Name': 5}


In [148]:
print("Filtering to %s of total races"%(len(selection)*100.0/total_headers_parsed)) 

Filtering to 59.58468710269812 of total races


In [7]:
import pandas as pd

def extract_results_from_page(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    all_rows = soup.find_all('tr')
    results = []
    headers = []
    for row in all_rows:
        header_data = [cell.get_text(strip=True) for cell in row.find_all('th')]
        results_data = [cell.get_text(strip=True) for cell in row.find_all('td')]
        if len(header_data) > 0:
            headers = header_data
        elif len(headers) and len(headers) == len(results_data):
            results.append(results_data), headers
    return pd.DataFrame(results, columns=headers)

limit_processing = 100000000
count = 0
race_results_home = "race_results"
if not(os.path.exists(race_results_home)):
    os.makedirs(race_results_home)
    
for race_path in os.listdir("temp_page_cache"):
    if count > limit_processing: break
    # Iterate over directories to get race name, then sub directories to get date, 
    # and then files in directorises to get results
    output_path = os.path.join(race_results_home, race_path+".parquet")
    if os.path.exists(output_path):
        continue
        
    race_results = None
    for date in os.listdir(os.path.join("temp_page_cache", race_path)):
        if count > limit_processing: break
            
        for file_part in os.listdir(os.path.join("temp_page_cache", race_path, date)):
            if file_part == "headers.txt": continue
            count += 1
            if count > limit_processing: break

            target_source_path = os.path.join("temp_page_cache", race_path, date, file_part)
            if os.path.isdir(target_source_path):
                # This is because of some string corruption in the previos parsing stage, only for weird races so 
                # we discard
                print("Unable to parse: %s"%target_source_path)
                continue
            with gzip.open(os.path.join("temp_page_cache", race_path, date, file_part)t) as r_handle:
                source = r_handle.read().decode('utf-8')
                partial_race_results = extract_results_from_page(source)
                partial_race_results["date"] = date
                if race_results is None:
                    race_results = partial_race_results
                else:
                    race_results = pd.concat([race_results, partial_race_results], ignore_index=True)
    if race_results is not None:
        race_results["race"] = race_path
        race_results.to_parquet(output_path)


Unable to parse: temp_page_cache/caribou_marathon_(usa/me)/9_17_17
Unable to parse: temp_page_cache/caribou_marathon_(usa/me)/9_18_16
Unable to parse: temp_page_cache/caribou_marathon_(usa/me)/9_15_19
Unable to parse: temp_page_cache/caribou_marathon_(usa/me)/9_16_18
Unable to parse: temp_page_cache/walker/north_country_races/9_20_14
Unable to parse: temp_page_cache/walker/north_country_races/9_17_11
Unable to parse: temp_page_cache/walker/north_country_races/9_21_02
Unable to parse: temp_page_cache/walker/north_country_races/9_22_01
Unable to parse: temp_page_cache/walker/north_country_races/9_18_10
Unable to parse: temp_page_cache/walker/north_country_races/9_17_16
Unable to parse: temp_page_cache/walker/north_country_races/9_15_12
Unable to parse: temp_page_cache/walker/north_country_races/9_27_03
Unable to parse: temp_page_cache/walker/north_country_races/9_21_13
Unable to parse: temp_page_cache/walker/north_country_races/9_16_06
Unable to parse: temp_page_cache/walker/north_countr